In [1]:
import numpy as np
import shapely
import bisect
import shapely.wkt
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import json
import pandas as pd
import seaborn as sns
from enum import Enum


from bench_utils import parse_intersection_data 
from algos.fpd_extended_lib.cfg import MAX_NUM_DELTAS 

In [2]:
# Geometries
manual_data, _ = parse_intersection_data("manual")
special_cases, _ = parse_intersection_data("latest_export.json", strip_precision=True)
world_data, world_data_stats = parse_intersection_data("world.json", 10, strip_precision=False)
lund_data, lund_data_stats = parse_intersection_data("lund.json", 100)

In [3]:
SHOW_COORDINATES = True
SHOW_GEOMETRIES = True
SHOW_INTERSECTING_POINTS = True
SHOW_COMMON_BOUNDING_BOX = False
SHOW_BOUNDING_BOXES = False
SHOW_STATS = True

LARGE_THRESHOLD = 50
MAX_NUM_DELTAS = 1


In [4]:
import intersection.first_bin_search
import intersection.chunk_bbox_intersection
binary_intersection = intersection.first_bin_search.binary_intersection
chunk_bbox_is_intersecting = intersection.chunk_bbox_intersection.is_intersecting
chunk_bbox_intersection = intersection.chunk_bbox_intersection.intersection

from intersection.plotting import *

from algos.alg_fpd_extended import FpdExtended

fpd = FpdExtended()

In [5]:
total = 0
passed = 0
stats_df = pd.DataFrame(columns=['decomp', 'nbr_recieved_chks', 'nbr_total_chks', 'total_time', "dataset", "context", "predicate"])

def evaluate_dataset(data, plot, total, passed, dataset_name):
    for g1, g2 in data:
        #is_intersecting, intersect_points = binary_intersection(g1, g2)
        _, b1 = fpd.compress(g1)
        _, b2 = fpd.compress(g2)
        exp_boolean = shapely.intersects(g1, g2)
        #is_intersecting = chunk_bbox_is_intersecting((b1, b2), exp_boolean, True)
        stats_is_intersection, is_intersecting = chunk_bbox_is_intersecting((b1, b2), exp_boolean, True, get_stats=True)

        exp_shape = shapely.intersection(g1, g2)
        stats_intersection, intersection = chunk_bbox_intersection((b1, b2), exp_boolean, True, get_stats=True)
        if plot and exp_boolean: # Now only plotting intersecting shapes
            if False: # Plot chunks?
                plot_chunks_bounds(b1, True, avoid_show=True)
                plot_chunks_bounds(b2, True, avoid_create_frame=True, avoid_show=True)
            geoms = (g1, g2)
            for g in geoms:
                plot_geometry(g, SHOW_GEOMETRIES)
                plot_geometry_bbox(g, SHOW_BOUNDING_BOXES)
                plot_coordinates(g, SHOW_COORDINATES)
            
            plot_common_bbox(geoms, SHOW_COMMON_BOUNDING_BOX)
            #plot_intersecting_points(intersect_points, legends, SHOW_INTERSECTING_POINTS)

            if SHOW_COORDINATES or SHOW_GEOMETRIES or SHOW_INTERSECTING_POINTS or SHOW_COMMON_BOUNDING_BOX or SHOW_BOUNDING_BOXES:
                plt.title("Intersection Plot: " + ('True' if is_intersecting else 'False'))
                plt.show()


            PLOT_RESULTING_INTERSECTION = True
            if is_intersecting and PLOT_RESULTING_INTERSECTION:
                create_canvas(zoom=1.1, no_frame=True)
                for g in geoms:
                    plot_geometry(g, SHOW_GEOMETRIES, alpha=0.2, fill_alpha=0.2)
                    plot_coordinates(g, SHOW_COORDINATES, size=1)
                
                plot_geometry(intersection, fill_alpha=0.6, hatch=True)
                plot_coordinates(intersection, size=10)
                #plt.title("Intersecting Shape")
                plt.show()

        total += 1

        #Add statistics to stats_df
        if SHOW_STATS:
            for idx,stats in enumerate([stats_is_intersection, stats_intersection]):
                stats.append(dataset_name)
                s1, s2 = (shapely.get_num_coordinates(g1), shapely.get_num_coordinates(g2))
                sz = "MIXED"
                if s1 < LARGE_THRESHOLD and s2 < LARGE_THRESHOLD:
                    sz = "SMALL"
                elif s1 >= LARGE_THRESHOLD and s2 >= LARGE_THRESHOLD:
                    sz = "LARGE"
                
                stats.append(sz)
                stats.append(False if idx == 1 else True)
                stats_df.loc[len(stats_df)] = stats

        
        if is_intersecting == exp_boolean and exp_shape.equals(intersection):
            passed += 1
        #     if not exp_shape.is_empty:
        #         create_canvas()
        #         plot_geometry(g1,alpha=0.2)
        #         plot_geometry(g2,alpha=0.2)
        #         #plot_geometry(exp_shape, solid=False)
        #         plot_geometry(intersection)
        #         plot_coordinates(intersection)
        #         plt.show()
            
        #else:

        #     print("Above FAILED!")
        #     print('Expected shape:', exp_shape)
        #     print('Returned shape:', intersection)
        #     create_canvas()
        #     plot_geometry(g1,alpha=0.2, fill_alpha=0.2, solid=False)
        #     plot_geometry(g2,alpha=0.2, fill_alpha=0.2, solid=False)
        #     #plot_geometry(exp_shape, solid=False)
        #     plot_geometry(intersection, fill_alpha=1.0)
        #     plot_coordinates(intersection)
        #     plt.show()
        # if total % 100 == 0:
        #     print(f"Passed {passed} of {total}. Total in set: {len(data)}")
    return total, passed

#total, passed = evaluate_dataset(manual_data, False, total, passed, "Manual") # Second param: PLOT?

total, passed = evaluate_dataset(special_cases, False, total, passed, "Special Case")
total, passed = evaluate_dataset(world_data, False, total, passed, "World")
total, passed = evaluate_dataset(lund_data, False, total, passed, "Lund")

print(f'Done. Passed: {passed} of {total}.')
#POLYGON ((6.5 6.5, 8 5, 4 4, 6.5 6.5))

Done. Passed: 119 of 121.


In [13]:
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go

if SHOW_STATS:
    for idx, df in enumerate([stats_df[stats_df['predicate'] == True], stats_df[stats_df['predicate'] == False]]): #Loop through predicate and not predicate intersection seperately
        #Calculate decompression and intersection time fraction
        df["decomp"] = df["decomp"].div(df.total_time/100, axis=0);
        df['intersection' if idx == 1 else 'is_intersection'] = df.apply(lambda x: 100 - x['decomp'], axis=1);
        exec_fractions_df = df[["decomp", 'intersection' if idx == 1 else 'is_intersection', "dataset", "context"]];
        exec_fractions_df = exec_fractions_df.groupby(['context','dataset']).mean();
        exec_fractions_df.reset_index(inplace=True)
       
        x = [
            exec_fractions_df.context.values,
            exec_fractions_df.dataset.values
        ]
        fig = go.Figure()
        fig.add_bar(x=x,y=exec_fractions_df.decomp.values, name="Partial decompression")
        if idx == 0:
            fig.add_bar(x=x,y=exec_fractions_df.is_intersection.values, name="IsIntersection")
            fig.update_layout(title="Execution time distrubution for IsIntersection")

        else:
            fig.add_bar(x=x,y=exec_fractions_df.intersection.values, name="Intersection")
            fig.update_layout(title="Execution time distrubution for Intersection")
        fig.update_layout(barmode="relative", yaxis_title="Execution time (%)", xaxis_title="Context (Geometry Size + Dataset)")

        fig.show()

In [15]:
if SHOW_STATS:
    for idx, df in enumerate([stats_df[stats_df['predicate'] == True], stats_df[stats_df['predicate'] == False]]): #Loop through predicate and not predicate intersection seperately
        #Calculate franctions of mean fraction of chunks unfolded
        df["chk_fraction_unfolded"] = df["nbr_recieved_chks"].div(df.nbr_total_chks, axis=0) * 100;
        chk_fractions_df = df[["chk_fraction_unfolded", "dataset", "context"]];
        chk_fractions_df = chk_fractions_df.groupby(['context','dataset']).mean();
        chk_fractions_df.reset_index(inplace=True)
       
        x = [
            chk_fractions_df.context.values,
            chk_fractions_df.dataset.values
        ]
        fig = go.Figure()
        fig.add_bar(x=x,y=chk_fractions_df.chk_fraction_unfolded.values, name="Chunk fraction unfolded")
        
        if idx == 0:
            fig.update_layout(title="Fraction of chunks unfolded for IsIntersection")
        else:
            fig.update_layout(title="Fraction of chunks unfolded for Intersection")
        fig.update_layout(barmode="relative", yaxis_title="Chunks unfolded (%)", xaxis_title="Context (Geometry Size + Dataset)")

        fig.show()


In [8]:
#Ongoing chunk analysis

import algos.fpd_extended_lib.cfg as cfg
d_cnt_size_df = pd.DataFrame(columns=['d_cnt_size', 'decomp', 'nbr_recieved_chks', 'nbr_total_chks', 'total_time', "dataset", "context", "predicate"])

for i in [1]:
    cfg.MAX_NUM_DELTAS = i
    stats_df = pd.DataFrame(columns=['decomp', 'nbr_recieved_chks', 'nbr_total_chks', 'total_time', "dataset", "context", "predicate"])
    total, passed = evaluate_dataset(special_cases, False, total, passed, "Special Case")
    total, passed = evaluate_dataset(world_data, False, total, passed, "World")
    total, passed = evaluate_dataset(lund_data, False, total, passed, "Lund")
    display(stats_df.groupby(['dataset', 'context','predicate']).mean().reset_index())
    #stats_df
    

,dataset,context,predicate,decomp,nbr_recieved_chks,nbr_total_chks,total_time
0,Lund,MIXED,False,0.000382,3.700000,33.000000,0.000999
1,Lund,MIXED,True,0.000367,3.600000,33.000000,0.000635
2,Lund,SMALL,False,0.000115,2.144444,2.255556,0.000274
3,Lund,SMALL,True,0.000127,2.144444,2.255556,0.000254
4,Special Case,LARGE,False,0.003357,67.000000,90.000000,0.016771
5,Special Case,LARGE,True,0.002128,28.000000,90.000000,0.003718
6,Special Case,MIXED,False,0.000206,3.000000,13.500000,0.001647
7,Special Case,MIXED,True,0.000183,2.500000,13.500000,0.000462
8,Special Case,SMALL,False,0.000349,5.875000,7.875000,0.004954
9,Special Case,SMALL,True,0.000335,5.625000,7.875000,0.000767
